Simple Gen AI App Using LangChain

In [11]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [12]:
## DATA INGESTION -- FROM the Website we need to scrape the data 

from langchain_community.document_loaders import WebBaseLoader 

In [13]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/self-hosted")
loader

In [14]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/self-hosted', 'title': 'Self-hosted LangSmith - Docs by LangChain', 'language': 'en'}, page_content='Self-hosted LangSmith - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationSelf-hostedSelf-hosted LangSmithGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewSet up LangSmithCloud (SaaS)HybridOverviewSetup guideSelf-hostedOverviewGet started by cloud providerSetup guidesEnable featuresConfigurationConnect external servicesPlatform auth & access controlSelf-hosted observabilityScripts for management tasksOn this pageSelf-host LangSmith observability and evaluationServicesStorage servicesSetup methodsSetup guidesEnable LangSmith DeploymentWorkflowStandalone ServerWorkflowSupported compute platformsSetup guideSelf-hostedSelf-hosted LangSmithCopy pageCopy pageImportant\nSelf-hosted LangSmith is

In [27]:
## Load Data --> Docs --> Divide our text into chunks -->vectors-->Vector Embeddings --> Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [28]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/self-hosted', 'title': 'Self-hosted LangSmith - Docs by LangChain', 'language': 'en'}, page_content='Self-hosted LangSmith - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationSelf-hostedSelf-hosted LangSmithGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewSet up LangSmithCloud (SaaS)HybridOverviewSetup guideSelf-hostedOverviewGet started by cloud providerSetup guidesEnable featuresConfigurationConnect external servicesPlatform auth & access controlSelf-hosted observabilityScripts for management tasksOn this pageSelf-host LangSmith observability and evaluationServicesStorage servicesSetup methodsSetup guidesEnable LangSmith DeploymentWorkflowStandalone ServerWorkflowSupported compute platformsSetup guideSelf-hostedSelf-hosted LangSmithCopy pageCopy pageImportant'),
 Document(metadata={'s

In [29]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [31]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [32]:
vectorstoredb

In [ ]:
## Query from a vector store db
query="LangSmith supports different self-hosted configurations depending on your scale"
result = vectorstoredb.similarity_search(query)
result[0].page_content

In [34]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")


In [47]:
# Retrieval Chain , Document Chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based on the provided context:
<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based on the provided context:\n<context>\n{context}\n</context>\n\nQuestion: {input}'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.res

In [49]:
from langchain_core.documents import Document

document_chain.invoke({
    "input" : "Langsmith has two usage limits : total traces and extended ",
    "context" : [Document(page_content="LangSmith has two usage limits : total traces and extended traces per month. Total traces are the total number of traces that can be created in a month, while extended traces are the number of traces that can be created with additional features such as longer retention and more detailed information.")]
})

'traces per month. The total traces limit refers to the overall number of traces that can be created within a month. In contrast, the extended traces limit pertains to the number of traces that can be created with additional features, such as longer retention and more detailed information.'

However, we want the document to first come from the retriever we just set up. That way, we can use the retriever to dynamically 
select the most relevant documents and pass those in for a given question 

In [50]:
### Retriever 
vectorstoredb

In [54]:
retriever = vectorstoredb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [61]:
## Get response from the llm
response = retrieval_chain.invoke({"input":"Langsmith has two usage limits : total traces and extended"})

In [62]:
response['context']

[Document(id='8b738a9a-b5c9-4f5c-b461-318cb134d965', metadata={'source': 'https://docs.langchain.com/langsmith/self-hosted', 'title': 'Self-hosted LangSmith - Docs by LangChain', 'language': 'en'}, page_content='ModelIncludesBest forMethodsObservability & EvaluationLangSmith (UI + API)Backend services (queue, playground, ACE)Datastores: PostgreSQL, Redis, ClickHouse, optional blob storageTeams who need self-hosted observability, tracing, and evaluationRunning LangSmith without deploying agents/graphsDocker Compose (dev/test)Kubernetes + Helm (production)Observability, Evaluation & DeploymentEverything from Observability and EvaluationControl plane (deployments UI, revision management, Studio)Data plane (Agent Server pods)Kubernetes operator for orchestrationEnterprise teams needing a private LangChain CloudCentralized UI/API for managing multiple agents/graphsIntegrated observability and orchestrationKubernetes with Helm (required)Runs on EKS, GKE, AKS, or self-managed clustersStandalo